In [ ]:
# We might use this array to set values on specific features
constraints = pd.DataFrame({'constrained_feature': ["X1", "X3"], 'constrained_feature_value': [-1, 4]}).set_index("constrained_feature")

# We define the number of individuals at each generation and the selected number
generation_size = 100
population_out_size = 10

# We initiate the 1st population, based on the original dataset features
starting_population = generate_min_max_population(df.drop("Y", axis=1), constraints, generation_size)
features_names = starting_population.columns

# Target is set
target = 42

# We set the number of successive generations
generation_nb = 5000

# We set a variable to record the total number of individuals reached at each stage
individuals_nb = 0

# We define a variable to record every improvement on the target distance
memory  = 100

# We create a DataFrame to record the min target from distance at each iteration
results_min_max = pd.DataFrame(np.zeros((generation_nb,3)), columns=["min_target_distance","individuals_nb","time_elapsed_min_max"])

start_timer = time.time()

for i in range(generation_nb):
  
  # We either initiate the loop with the starting or previous population
  if i==0:
    population_in = starting_population
  else:
    population_in = population_out.drop(["Y","target_distance"], axis=1)
  
  # A new generation is created and only the best individuals are returned
  population_out = min_max_select(constraints, population_in, features_names, generation_size, population_out_size, target, model)

  # The current minimum distance from target is set and recorded
  current_min = population_out.iloc[0,population_out.shape[1]-1]
  results_min_max.loc[i,"min_target_distance"] = current_min

  # The incremental number of individuals created is calculated and recorded
  individuals_nb+=generation_size
  results_min_max.loc[i,"individuals_nb"] = individuals_nb


  results_min_max.loc[i,"time_elapsed_min_max"] = float(time.time()-start_timer)

  # In case there is a improvement on the minimum distance, we display it
  if current_min<memory:
    memory = current_min
    print(i, ":", memory)
  i+=1